In [ ]:
from random import random
import os
import shutil

def copy_file(source_directory, destination_directory, filename):
    """
    Utility function used to copy a file from a source_directory to a destination_directory
    """
    destination_directory.mkdir(parents=True, exist_ok=True)
    shutil.copy(source_directory/filename, destination_directory/filename)
    
def organize_train_valid_dataset(root, dest, labels, valid_probability=0.1):
    """
    Creates the train, train_valid and valid folders respecting PyTorch's ImageDataset structure, performing
    train/validation split based on the given percentage
    """
    source_directory = root/'images'
    
    for index, row in labels.iterrows():
        img_index = row['filename'].split('.')[0]  # The filename is the name of the image except the extension
        img_class = row['quantity']

        # Randomly assign the image to the valid dataset with probability 'valid_probability'
        channel = Path('train') if random()>valid_probability else Path('valid')
        destination_directory = dest/channel/str(img_class)

        # Copy the image to either the train or valid folder, and also to the train_valid folder
        copy_file(source_directory, destination_directory, row['filename'])
        copy_file(source_directory, dest/'train_valid'/str(img_class), row['filename'])

def organize_test_dataset(root, dest, labels):
    """
    Creates the test folder respecting PyTorch's ImageDataset structure, using a dummy 'undefined' label
    """
    source_directory = root/'images'
        
    for index, row in labels.iterrows():
        img_index = row['filename'].split('.')[0]  # The index is the name of the image except the extension

        channel = Path('test')
        destination_directory = dest/channel/'undefined'
        
        try:
            copy_file(source_directory, destination_directory, row['filename'])
        except Exception as e:
            print(e)

In [ ]:
import pandas as pd
from pathlib import Path


root = Path('.')
dest = Path('.')
dest.mkdir(exist_ok=True, parents=True)

# Read in the labels DataFrame with a label for each image
train = pd.read_csv(f'{root}/train.csv')
test = pd.read_csv(f'{root}/test.csv')

# Create the train/train_valid/valid folder structure
valid_probability = 0.1
organize_train_valid_dataset(root, dest, train, valid_probability)

# Create the test folder structure
organize_test_dataset(root, dest, test)

In [ ]:
# dataloader

In [ ]:
import torch, torchvision
from torch import nn

train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
    ])

train_dataset, train_valid_dataset = [torchvision.datasets.ImageFolder(folder, transform=train_transforms) for folder in [root/'train', root/'train_valid']]


valid_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
    ])

valid_dataset, test_dataset = [torchvision.datasets.ImageFolder(folder, transform=valid_transforms) for folder in [root/'valid', root/'test']]

In [ ]:
## Trying to balance the data -- in progress

def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight                                                                         
                                                                                
# For unbalanced dataset we create a weighted sampler                       
#weights = make_weights_for_balanced_classes(train_dataset.imgs, 6)                                                                
#weights = torch.DoubleTensor(weights)                                       
#sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))                     
                                                                                
#train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, sampler = sampler, num_workers=2*num_gpus, pin_memory=True) 

#train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2*num_gpus, pin_memory=True)
#train_valid_dataloader = torch.utils.data.DataLoader(train_valid_dataset, batch_size=128, shuffle=True, num_workers=2*num_gpus, pin_memory=True)

#valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=2*num_gpus, pin_memory=True)
#test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2*num_gpus, pin_memory=True)

In [ ]:
num_gpus = torch.cuda.device_count()
print(num_gpus)

In [ ]:
# create dataloaders

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2*num_gpus, pin_memory=True)
train_valid_dataloader = torch.utils.data.DataLoader(train_valid_dataset, batch_size=128, shuffle=True, num_workers=2*num_gpus, pin_memory=True)

valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=2*num_gpus, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2*num_gpus, pin_memory=True)

In [ ]:
def get_net():
    resnext = torchvision.models.resnext50_32x4d(pretrained=True)
    num_ftrs = resnext.fc.in_features
    #resnext.fc = torch.nn.Linear(num_ftrs, 6)
    #resnext.classifier[6] = torch.nn.Linear(resnext.classifier[6].in_features, 6)
    resnext.fc = nn.Sequential(
        nn.BatchNorm1d(num_ftrs),
        nn.Linear(in_features=num_ftrs, out_features=512, bias=False),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(0.4),
        nn.Linear(in_features=512, out_features=6, bias=False)
    )
    #torch.nn.init.xavier_uniform_(resnext.fc.weight) # initialize the weights of the new layer
    return resnext

In [ ]:
#!pip install timm

In [ ]:
## trying some other model -- not working

#import timm
#def get_net_exp():
#    convnext_tiny = timm.create_model('convnext_small_in22ft1k', pretrained=True)
#    num_ftrs = convnext_tiny.head.fc.in_features
#    convnext_tiny.head.fc = torch.nn.Linear(in_features=num_ftrs, out_features=6, bias=True)
    
    #resnext = torchvision.models.vit_b_16(pretrained=True)
    
    #resnext.fc = torch.nn.Linear(num_ftrs, 6)
    #resnext.classifier[6] = torch.nn.Linear(resnext.classifier[6].in_features, 6)
#    torch.nn.init.xavier_uniform_(convnext_tiny.head.fc.weight) # initialize the weights of the new layer
#    return convnext_tiny

In [ ]:
#for name, child in resnext.named_children():
#    print(name)

In [ ]:
#convnext_base = timm.create_model('convnext_base', pretrained=True)

In [ ]:
#convnext_base = timm.create_model('convnext_base', pretrained=True)

In [ ]:
#timm.list_models('*convnext*')

In [ ]:
import time

def train(net, train_dataloader, valid_dataloader, criterion, optimizer, scheduler=None, epochs=10, device='cpu', checkpoint_epochs=10):
    start = time.time()
    print(f'Training for {epochs} epochs on {device}')
    
    for epoch in range(1,epochs+1):
        print(f"Epoch {epoch}/{epochs}")
        
        net.train()  # put network in train mode for Dropout and Batch Normalization
        train_loss = torch.tensor(0., device=device)  # loss and accuracy tensors are on the GPU to avoid data transfers
        train_accuracy = torch.tensor(0., device=device)
        for X, y in train_dataloader:
            X = X.to(device)
            y = y.to(device)
            preds = net(X)
            loss = criterion(preds, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_loss += loss * train_dataloader.batch_size
                train_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if valid_dataloader is not None:
            net.eval()  # put network in train mode for Dropout and Batch Normalization
            valid_loss = torch.tensor(0., device=device)
            valid_accuracy = torch.tensor(0., device=device)
            with torch.no_grad():
                for X, y in valid_dataloader:
                    X = X.to(device)
                    y = y.to(device)
                    preds = net(X)
                    loss = criterion(preds, y)

                    valid_loss += loss * valid_dataloader.batch_size
                    valid_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        
        if scheduler is not None: 
            scheduler.step()
            
        print(f'Training loss: {train_loss/len(train_dataloader.dataset):.2f}')
        print(f'Training accuracy: {100*train_accuracy/len(train_dataloader.dataset):.2f}')
        
        if valid_dataloader is not None:
            print(f'Valid loss: {valid_loss/len(valid_dataloader.dataset):.2f}')
            print(f'Valid accuracy: {100*valid_accuracy/len(valid_dataloader.dataset):.2f}')
        
        if epoch%checkpoint_epochs==0:
            torch.save({
                'epoch': epoch,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, './checkpoint.pth.tar')
        
        print()
    
    end = time.time()
    print(f'Total training time: {end-start:.1f} seconds')
    return net

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr, epochs = 1e-3, 10

# get network and move it to GPU
net = get_net().to(device)

# standard CrossEntropy Loss for multi-class classification problems
criterion = torch.nn.CrossEntropyLoss()

# define the optimizer for the network parameters
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# start training process
net = train(net, train_dataloader, valid_dataloader, criterion, optimizer, None, epochs, device)